# APIs

## Introduction

APIs are the building block of the web as we know it. It allows users and PCs to communicate over a set of standards and protocols. This notebook introduces the concepts and shows how you can create an API using Python.

Let start at the beginning. What does API stand for? Well, Application Programming Interface. APIs come in two different flavours, namely Simple Object Access Protocol or SOAP, and Representational State Transfer or REST. REST is far more common and SOAP is a bit old aged so we'll just focus on REST APIs in this notebook.

<img src='https://process.filestackapi.com/cache=expiry:max/Pbm6DJWRBSgfuf0TfLUg'>

Most, if not all, large popular websites will rely upon some form of REST API in order to deliver some content or functionality to their users. Some sites like Facebook and Twitter actually expose some of these APIs to outside developers to build their own tools and systems.

We can communicate with REST APIs using HTTP requests, much like you’d do to navigate to a website or load an image. We can do HTTP requests to certain API urls and these urls would then return the information we required, or we could push data to an API url in order to change some data in a database.

Typically we send HTTP requests to an URL that we have defined in our REST API and it would either perform a given task for us or return a certain bit of data. Most APIs these days would return a response to us in the form of JSON.

<img src='assets/rest-api.png'>

The introduction above a most of the examples below are taken word for word from <a href='https://tutorialedge.net/general/what-is-a-rest-api/'>here</a>.

## Example

Imagine you wrote a bit of code that gives you the current weather conditions at your house. It reads the temperature, humidity and rainfall and stores them locally. How would we then expose this information in such a way that websites or other applications could view it?

One answer to this question is by wrapping it in a RESTful API.

We could expose our code and wrap it in an API so that whenever we navigated to say http://localhost:8000/api/weatherStats it would give us a JSON response that contained all the current weather stats.

## Why do we do this?

**Improved Code Reuse** - By exposing our code through REST APIs we essentially give ourselves a greater degree of flexibility. We can develop our software once and should we wish to use the same code again in a different project it would be easy, we could simply send HTTP requests to our API and we’ve reduced the need to duplicate our work efforts.

**Always Available** - REST APIs are typically things that are running and available all the time. We make them very stable and as a result we can interact with them wherever we are in the world as long as we have internet connectivity.

## Implematation in Python

### Querying an existing API

Example code and text taken from <a href='https://www.dataquest.io/blog/python-api-tutorial/'>here</a>. When we want to receive data from an API, we need to make a **request**. Requests are used all over the web. For instance, when you visit a webpage, your web browser makes a request to the web server, which responded with the content of the web page.

API requests work in exactly the same way – you make a request to an API server for data, and it responds to your request.

To make requests in python we will need the **requests** module.

In [1]:
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import requests

There are many different types of requests. The most commonly used one, a **GET** request, is used to retrieve data. Because we’ll just be working with retrieving data, our focus will be on making ‘get’ requests.

<img src='https://www.dataquest.io/wp-content/uploads/2019/09/api-request.svg'>

When you make a request and the server responds, you don't just get what you asked for, you also get a Response Code telling you something abou the state of you transaction. 

To make a ‘GET’ request, we’ll use the `requests.get()` function, which requires one argument — the URL we want to make the request to. We’ll start by making a request to an API endpoint that doesn’t exist, so we can see what that response code looks like.

In [4]:
response = requests.get("http://api.open-notify.org/this-api-doesnt-exist")

In [5]:
response.status_code

404

#### API Status Codes
Status codes are returned with every request that is made to a web server. Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

- 200: Everything went okay, and the result has been returned (if any).
- 301: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
- 400: The server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
- 401: The server thinks you’re not authenticated. Many APIs require login ccredentials, so this happens when you don’t send the right credentials to access an API.
- 403: The resource you’re trying to access is forbidden: you don’t have the right permissions to see it.
- 404: The resource you tried to access wasn’t found on the server.
- 503: The server is not ready to handle the request.

You might notice that all of the status codes that begin with a ‘4’ indicate some sort of error. The first number of status codes indicate their categorization. This is useful — you can know that if your status code starts with a ‘2’ it was successful and if it starts with a ‘4’ or ‘5’ there was an error. If you’re interested you can read more about status codes

#### Documentation 

In order to know what API end points you can speak to you'll need to read the APIs documentation. For our example we'll be using the http://open-notify.org/ API which is an open source project to provide a simple programming interface for some of NASA’s awesome data. 

Let's see how the documentation looks like: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

So if we make request to the endpoint: http://api.open-notify.org/iss-now.json we will get the live location of the ISS. Let's give it a go.

In [6]:
response = requests.get('http://api.open-notify.org/iss-now.json')

In [9]:
response.status_code

200

`200` looks good! 

In [10]:
response.content

b'{"iss_position": {"longitude": "129.9716", "latitude": "-51.2891"}, "message": "success", "timestamp": 1572347245}'

Note the `b'` before the content, this tells us that this data is in `byte` format and has not been decoded yet.

In [12]:
type(response.content)

bytes

We can easily decode it by using the `decode()` method and specifying `utf-8` as this is the most widely used encoding/decoding standard.

In [13]:
response.content.decode('utf-8')

'{"iss_position": {"longitude": "129.9716", "latitude": "-51.2891"}, "message": "success", "timestamp": 1572347245}'

Now we have a string, but we would much rather like this in a dictionary. You can see that the string is basically in the right format, we just need to parse it. This is made very simple using the `json` library's `loads()` function

In [15]:
import json

In [16]:
json.loads(response.content.decode('utf-8'))

{'iss_position': {'longitude': '129.9716', 'latitude': '-51.2891'},
 'message': 'success',
 'timestamp': 1572347245}

Let's make a fun little program polling the ISS for it's position every second and plotting it. I'll be using `pandas` to create a dataframe holding all the points and using `plotly express` to plot the points. To make it a bit cleaner I'm using the `IntProgress` class to display a progress bar in the notebook.

In [39]:
from time import sleep
import pandas as pd
import plotly.express as px

from ipywidgets import IntProgress
from IPython.display import display

In [42]:
points = 10
df = pd.DataFrame()
f = IntProgress(min=0, max=points)
display(f)

for i in range(points):
    f.value += 1
    response = requests.get('http://api.open-notify.org/iss-now.json')
    resp = json.loads(response.content.decode('utf-8'))
    _ = resp['iss_position']
    _['timestamp'] =  resp['timestamp']
    df = pd.concat([df, pd.DataFrame(_, index=[1])])
    sleep(1)

df.longitude = df.longitude.astype(float)
df.latitude = df.latitude.astype(float)
df.head()

IntProgress(value=0, max=10)

,longitude,latitude,timestamp
1,-160.5092,-8.5117,1572348385
1,-160.4539,-8.4361,1572348387
1,-160.3803,-8.3353,1572348389
1,-160.3251,-8.2597,1572348390
1,-160.2700,-8.1841,1572348392


In [44]:
fig = px.scatter_mapbox(df,
                    lat='latitude',
                    lon='longitude',
                    animation_frame='timestamp')
fig.update_layout(mapbox_style="open-street-map",
              height=600,
              mapbox_zoom=5,
              title='ISS movement data',
              mapbox_center={"lat": df.latitude.mean(), "lon": df.longitude.mean()})
fig.show()

### Creating your own API

<a href='https://impythonist.wordpress.com/2015/07/12/build-an-api-under-30-lines-of-code-with-python-and-flask/'>This</a> is a great blog showing how you can hook up a SQLite database to your API, but we'll be creating an even simpler example.

#### Simple API

We'll be using `Flask` to build our API as it is a very powerful Python web framework. See <a href='https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask'>this</a> post for more details. I've created the simplest form of an API in the `apis` directory, title `api1.py`

In [46]:
!cat ../apis/api1.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8000)

In [48]:
!python ../apis/api1.py

 * Serving Flask app "api1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 278-273-286
127.0.0.1 - - [29/Oct/2019 13:40:49] "GET / HTTP/1.1" 200 -
^C


#### More advanced API

There are a couple of Flask extensions that help with building RESTful services with Flask.

The clients of our web service will be asking the service to add, remove and modify tasks, so clearly we need to have a way to store tasks. The obvious way to do that is to build a small database, but because databases are not the topic of this notebook we are going to take a much simpler approach.

In place of a database we will store our task list in a memory structure. This will only work when the web server that runs our application is single process and single threaded. This is okay for Flask's own development web server. It is not okay to use this technique on a production web server, for that a proper database setup must be used.

In [49]:
!cat ../apis/api2.py

#!flask/bin/python
from flask import Flask, jsonify

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]

@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

if __name__ == '__main__':
    app.run(debug=True)

In [51]:
!python ../apis/api2.py

 * Serving Flask app "api2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:8001/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 278-273-286
127.0.0.1 - - [29/Oct/2019 13:44:47] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2019 13:44:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2019 13:44:59] "GET /todo/api/v1.0/tasks HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2019 13:47:15] "GET /todo/api/v1.0/tasks HTTP/1.1" 200 -
^C


Now we can visit: http://127.0.0.1:8001/todo/api/v1.0/tasks and see our tasks. You might want to check out a REST client for you browser like <a href='https://addons.mozilla.org/en-US/firefox/addon/restclient/'>Rest Client</a> for firefox or <a href='https://chrome.google.com/webstore/detail/postman/fhbjgbiflinjbdggehcddcbncdddomop?hl=en'>Postman</a> for Chrome.

We can add a request for a specific task as shown in `api3.py`

In [52]:
!cat ../apis/api3.py

#!flask/bin/python
from flask import Flask, jsonify

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]

@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

if __name__ == '__main__':
    app.run(debug=True, host='127.0.0.1', port=8002)

In [53]:
!python ../apis/api3.py

 * Serving Flask app "api3" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:8002/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 278-273-286
127.0.0.1 - - [29/Oct/2019 13:49:14] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2019 13:49:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2019 13:49:18] "GET /todo/api/v1.0/tasks/1 HTTP/1.1" 200 -
^C


Now we can visit http://127.0.0.1:8002/todo/api/v1.0/tasks/1 to get the first task.